#  Rosenbrock function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (1)
---

The general equation is given by:

- $f(x, y) = (1 - x)^2 + 100(y - x^2)^2$, with $-1.5 \le x \le +1.5$ and $-1.5 \le y \le +1.5$.

The problem here is that we are trying to minimize this function subject to the following constraint:

- $x^2 + y^2 \le 2$.

To do this we apply the [Penalty method](https://en.wikipedia.org/wiki/Penalty_method). Within this setting the global minimum is found at:

$\hat{f}(1.0, 1.0) = 0.0$.

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.meta_mutator import MetaMutator

### Define the Rosebrock function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [3]:
# Rosenbrock function.
def fun_Rosenbrock(individual: Chromosome, f_min: bool = True):
    
    # Penalty coefficient.
    rho = 5.0
    
    # Extract the data values as 'x' and 'y', for parsimony.
    x, y = [gene.value for gene in individual.genome]
    
    # Compute the function value.
    f_val = (1.0 - x)**2 + 100.0*(y - x**2)**2
    
    # Compute the constraint.
    f_const = rho*max(0.0, x**2 + y**2 - 2.0)**2

    # Condition for termination.
    solution_found = np.isclose(f_val, 0.0, rtol=1.0e-4)

    # Assign the fitness value (check for minimization).
    fit_value = -(f_val + f_const) if f_min else f_val
    
     # NOTE: the constraint is added with the penalty coefficient.
    return fit_value, solution_found
# _end_def_

# Random function: ~U(-1.5, +1.5).
rand_fx = lambda: np.random.uniform(-1.5, +1.5)
rand_fy = lambda: np.random.uniform(-1.5, +1.5)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by 'rand_fx'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [4]:
# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.normal(), rand_fx),
                          Gene(np.random.normal(), rand_fy)], np.nan, True)
              for _ in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population,
                     fit_func=fun_Rosenbrock,
                     select_op=LinearRankSelector(),
                     mutate_op=MetaMutator(),
                     crossx_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter,
such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [5]:
test_GA(epochs=5000, elitism=True, adapt_probs=True)

Initial Avg. Fitness = -352.8507.
Final   Avg. Fitness = -30.0246.
Elapsed time: 110.041 seconds.


In [6]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Extract the fitness value from the optimal solution.
optimal_fit, _ = fun_Rosenbrock(optimal_solution, f_min=False)

# Display the (final) optimal value.
print(f"Minimum Found: {optimal_fit:.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.value:.5f}")
# _end_for_

# True minimum: f(1.0, 1.0) = 0.0

Minimum Found: 0.00000

x0 = 1.00003
x1 = 1.00003


### End of file